In [ ]:
# Import des bibliothèques principales
import pandas as pd  # Pour manipuler les données tabulaires
import numpy as np   # Pour les calculs numériques
import torch        # Framework de deep learning
import sys          # Pour modifier le path Python
import os           # Pour les opérations sur les fichiers

# Ajouter le dossier racine au path Python pour importer nos modules
sys.path.append('.')

print("Modules de base importés avec succès")
print("- pandas:", pd.__version__)
print("- numpy:", np.__version__)
print("- torch:", torch.__version__)


In [ ]:
# Import de tous les modules T4Rec depuis le dossier src
# Ces modules contiennent toute la logique du système de recommandation

from t4rec.data_generator import BankingDataGenerator          # Génère des données bancaires synthétiques
from t4rec.data_preprocessor import BankingDataPreprocessor    # Traite les données pour l'entraînement
from t4rec.transformer_model import BankingTransformerModel    # Modèle Transformer pour les recommandations
from t4rec.model_trainer import BankingModelTrainer            # Entraîne le modèle
from t4rec.recommendation_engine import RecommendationEngine   # Fait des prédictions

print("Librairie T4Rec importée avec succès")
print("Modules disponibles:")
print("- BankingDataGenerator (génération de données)")
print("- BankingDataPreprocessor (preprocessing)")
print("- BankingTransformerModel (modèle Transformer)")
print("- BankingModelTrainer (entraînement)")
print("- RecommendationEngine (inférence)")


In [ ]:
# Créer un générateur de données avec des paramètres réduits pour le test
# Configuration :
# - 50 clients uniques
# - 10 produits bancaires différents  
# - 100 sessions de navigation
# - seed=42 pour la reproductibilité

generator = BankingDataGenerator(
    n_customers=50,      # Nombre de clients simulés
    n_products=10,       # Nombre de produits bancaires (comptes, prêts, assurances...)
    n_sessions=100,      # Nombre de sessions de navigation
    seed=42              # Graine aléatoire pour des résultats reproductibles
)

print("BankingDataGenerator créé avec succès")
print(f"Configuration: {generator.n_customers} clients, {generator.n_products} produits, {generator.n_sessions} sessions")


In [ ]:
# Générer les données d'interactions bancaires
# Chaque ligne représente une interaction client-produit avec un timestamp

df = generator.generate_transactions()

print(f"Données générées: {len(df):,} transactions")
print(f"Colonnes créées: {list(df.columns)}")
print()
print("Structure des données:")
print("- customer_id: Identifiant du client")
print("- item_id: Produit bancaire consulté/acheté") 
print("- timestamp: Moment de l'interaction")
print("- session_id: Session de navigation")
print("- amount: Montant de la transaction")
print()
print("Aperçu des premières lignes:")
df.head()


In [ ]:
# Créer le preprocesseur avec une longueur de séquence maximale
# max_seq_length=6 signifie que chaque séquence aura exactement 6 éléments
# Les séquences plus courtes seront paddées avec des 0
# Les séquences plus longues seront tronquées

preprocessor = BankingDataPreprocessor(max_seq_length=6)
print("BankingDataPreprocessor créé")
print("Longueur max des séquences:", preprocessor.max_seq_length)

# Sauvegarder les données pour pouvoir les recharger
generator.save_data(df, "data/test_data.parquet")
print("Données sauvegardées dans data/test_data.parquet")

# Charger les données dans le preprocesseur
preprocessor.load_data("data/test_data.parquet")
print("Données chargées dans le preprocesseur")
print(f"Nombre de lignes chargées: {len(preprocessor.data)}")


In [ ]:
# Convertir les interactions en séquences par session
# Chaque session devient une séquence de produits consultés dans l'ordre chronologique

sequences = preprocessor.create_sequences()
print(f"Séquences créées: {sequences.shape}")
print("Forme: (nombre_sessions, longueur_séquence)")
print()

# Créer les paires input/target pour l'entraînement
# Input: séquence sans le dernier élément [A, B, C, D, E]
# Target: séquence sans le premier élément [B, C, D, E, F]
# Le modèle apprend à prédire le prochain élément

inputs, targets = preprocessor.create_training_pairs(sequences)
print(f"Paires d'entraînement créées:")
print(f"- Inputs: {inputs.shape} (séquences d'entrée)")
print(f"- Targets: {targets.shape} (séquences cibles)")
print()

# Afficher quelques exemples pour comprendre
print("Exemples de séquences (0 = padding):")
for i in range(3):
    print(f"Séquence {i+1}:")
    print(f"  Input:  {inputs[i].numpy()} -> Target: {targets[i].numpy()}")
    print(f"  Le modèle doit prédire chaque target à partir de l'input correspondant")


In [ ]:
# Créer le modèle Transformer avec des paramètres adaptés à nos données
# Configuration :
# - num_items=10 : nombre de produits bancaires (doit correspondre aux données)
# - embedding_dim=16 : dimension des représentations vectorielles (plus petit pour le test)
# - seq_length=6 : longueur des séquences (doit correspondre au preprocesseur)
# - num_heads=2 : nombre de têtes d'attention (plus petit pour le test)
# - num_layers=1 : nombre de couches Transformer (plus simple pour le test)

model = BankingTransformerModel(
    num_items=10,        # Nombre de produits bancaires
    embedding_dim=16,    # Taille des embeddings (représentations vectorielles)
    seq_length=6,        # Longueur des séquences
    num_heads=2,         # Têtes d'attention (parallélisation)
    num_layers=1         # Couches du Transformer
)

print(f"Modèle Transformer créé")
print(f"Nombre de paramètres: {model.count_parameters():,}")
print("Architecture:")
print(f"- Embedding dimension: {model.embedding_dim}")
print(f"- Nombre de têtes d'attention: {model.num_heads}")
print(f"- Nombre de couches: {model.num_layers}")

# Test rapide du modèle avec quelques séquences
# Un "forward pass" vérifie que le modèle fonctionne correctement
test_input = inputs[:5]  # Prendre 5 séquences pour tester
print(f"Test avec {test_input.shape[0]} séquences...")

with torch.no_grad():  # Pas de calcul de gradients pour le test
    output = model(test_input)
    
print(f"Test réussi: {test_input.shape} -> {output.shape}")
print("Le modèle transforme les séquences d'input en prédictions de produits")


In [ ]:
# Créer l'entraîneur avec le modèle et les hyperparamètres
# Configuration :
# - learning_rate=0.001 : vitesse d'apprentissage (standard)
# - batch_size=16 : nombre d'exemples traités ensemble (adapté à la taille des données)

trainer = BankingModelTrainer(
    model=model,            # Le modèle Transformer créé précédemment
    learning_rate=0.001,    # Taux d'apprentissage (contrôle la vitesse d'adaptation)
    batch_size=16          # Taille des lots pour l'entraînement
)

print("BankingModelTrainer créé")
print(f"Taux d'apprentissage: {trainer.learning_rate}")
print(f"Taille des batches: {trainer.batch_size}")
print(f"Optimiseur: {type(trainer.optimizer).__name__}")

# Lancer l'entraînement sur nos données
# Configuration de l'entraînement :
# - epochs=2 : nombre de passages sur toutes les données (réduit pour le test)
# - validation_split=0.2 : 20% des données réservées pour la validation

print()
print("Début de l'entraînement...")
print("Le modèle va apprendre à prédire le prochain produit dans une séquence")

trainer.train(
    inputs=inputs,              # Séquences d'entrée
    targets=targets,            # Séquences cibles
    epochs=2,                   # Nombre d'itérations complètes
    validation_split=0.2        # Proportion pour la validation
)

print()
print("Entraînement terminé")
print("Le modèle a appris les patterns dans les données bancaires")


In [ ]:
# Sauvegarder le modèle entraîné pour usage futur
# Le fichier contiendra tous les paramètres appris par le modèle

model_path = "models/test_model.pt"
trainer.save_model(model_path)

print(f"Modèle sauvegardé dans: {model_path}")
print("Ce fichier peut être chargé plus tard pour faire des prédictions")
print("sans avoir besoin de ré-entraîner le modèle")


In [ ]:
# Créer le moteur de recommandation en chargeant le modèle sauvegardé
# Il va automatiquement charger les paramètres appris pendant l'entraînement

engine = RecommendationEngine(model_path)
print("RecommendationEngine créé et modèle chargé")
print(f"Modèle chargé depuis: {model_path}")
print("Prêt à faire des prédictions sur de nouvelles séquences")

# Créer des séquences de test pour simuler différents profils de clients
# Chaque liste représente l'historique de produits consultés par un client
test_sequences = [
    [1, 3, 7],           # Client ayant consulté les produits 1, 3, puis 7
    [2, 5, 8, 4],        # Client avec un parcours plus long
    [6, 1],              # Client avec un parcours court
    [9, 3, 2, 7]         # Client avec un autre pattern
]

print()
print("Test de prédictions sur différents profils clients:")
print("Pour chaque séquence, le modèle prédit les 3 produits les plus probables")

# Tester le moteur de prédiction sur chaque séquence
for i, sequence in enumerate(test_sequences):
    print(f"")
    print(f"Test {i+1}: Client ayant consulté {sequence}")
    
    try:
        # Demander les 3 meilleures prédictions
        predictions = engine.predict(sequence, top_k=3)
        
        print(f"  Prédictions (top 3 produits recommandés):")
        for rank, (item_id, probability) in enumerate(predictions, 1):
            print(f"    {rank}. Produit {item_id} - Probabilité: {probability:.1%}")
            
    except Exception as e:
        print(f"  Erreur lors de la prédiction: {e}")

print()
print("Test terminé - Le système peut maintenant recommander des produits bancaires")
